In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import proplot as pplt

<https://www2.census.gov/geo/tiger/TIGER2021/CBSA/>

In [32]:
# boundary of MSAs with land/water area
msas = gpd.read_file("zip+https://www2.census.gov/geo/tiger/TIGER2021/CBSA/tl_2021_us_cbsa.zip")

In [33]:
msas

,CSAFP,CBSAFP,GEOID,NAME,NAMELSAD,LSAD,MEMI,MTFCC,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,122,12020,12020,"Athens-Clarke County, GA","Athens-Clarke County, GA Metro Area",M1,1,G3110,2654607902,26109459,+33.9439840,-083.2138965,"POLYGON ((-83.36003 34.04057, -83.36757 34.043..."
1,122,12060,12060,"Atlanta-Sandy Springs-Alpharetta, GA","Atlanta-Sandy Springs-Alpharetta, GA Metro Area",M1,1,G3110,22495873026,386782308,+33.6937280,-084.3999113,"POLYGON ((-84.27014 32.99101, -84.27084 32.991..."
2,428,12100,12100,"Atlantic City-Hammonton, NJ","Atlantic City-Hammonton, NJ Metro Area",M1,1,G3110,1438775279,301270067,+39.4693555,-074.6337591,"POLYGON ((-74.58640 39.30989, -74.58665 39.309..."
3,426,12120,12120,"Atmore, AL","Atmore, AL Micro Area",M2,2,G3110,2448595161,20024887,+31.1222867,-087.1684097,"POLYGON ((-87.36388 30.99790, -87.36391 30.997..."
4,258,12140,12140,"Auburn, IN","Auburn, IN Micro Area",M2,2,G3110,939731961,2657419,+41.3967596,-085.0026969,"POLYGON ((-85.07780 41.26560, -85.07850 41.265..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,None,49080,49080,"Winnemucca, NV","Winnemucca, NV Micro Area",M2,2,G3110,24969535441,44194837,+41.4079136,-118.1275915,"POLYGON ((-118.12404 41.99737, -118.12401 41.9..."
935,None,49100,49100,"Winona, MN","Winona, MN Micro Area",M2,2,G3110,1621791138,40051870,+43.9813526,-091.7770438,"POLYGON ((-91.71976 43.84694, -91.71983 43.846..."
936,268,49180,49180,"Winston-Salem, NC","Winston-Salem, NC Metro Area",M1,1,G3110,5203328303,81529764,+36.0788515,-080.3450642,"POLYGON ((-80.04324 36.01076, -80.04324 36.010..."
937,554,49220,49220,"Wisconsin Rapids-Marshfield, WI","Wisconsin Rapids-Marshfield, WI Micro Area",M2,2,G3110,2053959984,41993013,+44.4614126,-090.0388252,"POLYGON ((-89.72592 44.42309, -89.72588 44.421..."


Could also read directly from  <https://www2.census.gov/programs-surveys/popest/tables/2010-2019/metro/totals/cbsa-met-est2019-annres.xlsx>

In [10]:
# population data at MSA level
pops = pd.read_excel("../data/census/cbsa-met-est2019-annres.xlsx", skiprows=3)

In [12]:
pops = pops.rename(columns={'Unnamed: 0': 'name'})

In [19]:
pops.name = pops.name.str.replace('.', '')

/var/folders/79/cknfb1sx2pv16rztkpg6wzlw0000gn/T/ipykernel_28055/3438865337.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  pops.name = pops.name.str.replace('.', '')


In [22]:
msas = msas.merge(pops[['name', 2017]], left_on='NAMELSAD', right_on='name')

In [23]:
# make sure the 2017 column is converted to string
msas.columns = msas.columns.astype(str)

In [25]:
msas['pop_density'] = msas['2017']/msas['ALAND']

In [28]:
msas = msas[['GEOID', 'name', 'ALAND', 'AWATER', '2017', 'pop_density', 'geometry']]
msas = msas.rename(columns={'2017': 'population'})

In [29]:
msas

,GEOID,name,ALAND,AWATER,population,pop_density,geometry
0,12020,"Athens-Clarke County, GA Metro Area",2654607902,26109459,209106.0,0.000079,"POLYGON ((-83.36003 34.04057, -83.36757 34.043..."
1,12060,"Atlanta-Sandy Springs-Alpharetta, GA Metro Area",22495873026,386782308,5872432.0,0.000261,"POLYGON ((-84.27014 32.99101, -84.27084 32.991..."
2,12100,"Atlantic City-Hammonton, NJ Metro Area",1438775279,301270067,265498.0,0.000185,"POLYGON ((-74.58640 39.30989, -74.58665 39.309..."
3,12220,"Auburn-Opelika, AL Metro Area",1573526818,21516747,161440.0,0.000103,"POLYGON ((-85.16535 32.47097, -85.16725 32.470..."
4,12260,"Augusta-Richmond County, GA-SC Metro Area",9015018014,262412034,600276.0,0.000067,"POLYGON ((-81.65176 33.81451, -81.64701 33.818..."
...,...,...,...,...,...,...,...
380,48660,"Wichita Falls, TX Metro Area",6784839979,143918743,150956.0,0.000022,"POLYGON ((-98.42142 33.46680, -98.42140 33.465..."
381,48700,"Williamsport, PA Metro Area",3182886778,38917581,114012.0,0.000036,"POLYGON ((-76.96023 41.14880, -76.96126 41.145..."
382,48900,"Wilmington, NC Metro Area",2754582846,516637306,289447.0,0.000105,"POLYGON ((-77.67528 34.25313, -77.68032 34.250..."
383,49020,"Winchester, VA-WV Metro Area",2752456889,16817754,138020.0,0.000050,"POLYGON ((-78.34709 39.46601, -78.34704 39.465..."


In [30]:
msas.to_parquet("../data/census/msa_population.parquet")

In [31]:
msas.crs

<Geographic 2D CRS: EPSG:4269>
Name: NAD83
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: North America - onshore and offshore: Canada - Alberta; British Columbia; Manitoba; New Brunswick; Newfoundland and Labrador; Northwest Territories; Nova Scotia; Nunavut; Ontario; Prince Edward Island; Quebec; Saskatchewan; Yukon. Puerto Rico. United States (USA) - Alabama; Alaska; Arizona; Arkansas; California; Colorado; Connecticut; Delaware; Florida; Georgia; Hawaii; Idaho; Illinois; Indiana; Iowa; Kansas; Kentucky; Louisiana; Maine; Maryland; Massachusetts; Michigan; Minnesota; Mississippi; Missouri; Montana; Nebraska; Nevada; New Hampshire; New Jersey; New Mexico; New York; North Carolina; North Dakota; Ohio; Oklahoma; Oregon; Pennsylvania; Rhode Island; South Carolina; South Dakota; Tennessee; Texas; Utah; Vermont; Virginia; Washington; West Virginia; Wisconsin; Wyoming. US Virgin Islands. British Virgin Islands